In [1]:
%load_ext autoreload
%autoreload 2

import os
import re
import sys
from collections import defaultdict
from itertools import product
from pathlib import Path
from typing import Dict, List, Literal, Optional, Tuple, Union
from sklearn.metrics import  r2_score
import plotly.express as px
import scipy
from tqdm import tqdm
import IPython
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from matplotlib.axes import Axes as Axes
import logging
from time import perf_counter

NOTEBOOK_PATH:Path = Path(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"])
PROJECT_DIR:Path = NOTEBOOK_PATH.parent.parent
sys.path.append(str(PROJECT_DIR))
import src.utils.custom_log as custom_log
from src.utils.PathChecker import PathChecker
from src.utils.iso18571 import rating_iso_18571_short
from src.utils.set_rcparams import set_rcparams
from src._StandardNames import StandardNames

os.chdir(PROJECT_DIR)
LOG:logging.Logger = logging.getLogger(__name__)

custom_log.init_logger(log_lvl=logging.INFO)
LOG.info("Log initialized")

set_rcparams()

2024-08-06 12:52:43,448     INFO  MainProcess  MainThread Log initialized
2024-08-06 12:52:43,448     INFO  MainProcess  MainThread Setting rcparams for matplotlib
2024-08-06 12:52:43,484     INFO  MainProcess  MainThread Using style src/visualization/dissertation.mplstyle


In [2]:
STR: StandardNames = StandardNames()

DATA_DIR = PathChecker().check_directory(path=PROJECT_DIR / "data" / "doe" / "doe_big_grid_20230922_154140")
CHANNEL_FILE = PathChecker().check_file(path=DATA_DIR / STR.fname_channels)
FIG_DIR = PROJECT_DIR / "reports" / "figures" / NOTEBOOK_PATH.stem
FIG_DIR.mkdir(parents=True, exist_ok=True)
PathChecker().check_directory(path=FIG_DIR)

RELEVANT_CHANNELS: List[str] = [
    "03HEADLOC0OCCUDSXD",
    "03HEADLOC0OCCUDSYD",
    "03HEADLOC0OCCUDSZD",
    "03HEAD0000OCCUACXD",
    "03HEAD0000OCCUACYD",
    "03HEAD0000OCCUACZD",
    "03CHSTLOC0OCCUDSXD",
    "03CHSTLOC0OCCUDSYD",
    "03CHSTLOC0OCCUDSZD",
    "03CHST0000OCCUDSXD",
    "03CHST0000OCCUACXD",
    "03CHST0000OCCUACYD",
    "03CHST0000OCCUACZD",
    "03PELVLOC0OCCUDSXD",
    "03PELVLOC0OCCUDSYD",
    "03PELVLOC0OCCUDSZD",
    "03PELV0000OCCUACXD",
    "03PELV0000OCCUACYD",
    "03PELV0000OCCUACZD",
    "03NECKUP00OCCUFOXD",
    "03NECKUP00OCCUFOZD",
    "03NECKUP00OCCUMOYD",
    "03FEMRRI00OCCUFOZD",
    "03FEMRLE00OCCUFOZD",
]

TIME: np.ndarray = np.linspace(0, 140, 1401)

2024-08-06 12:52:43,520     INFO  MainProcess  MainThread Directory '/root/py_projects/aihiii/data/doe/doe_big_grid_20230922_154140' exists
2024-08-06 12:52:43,521     INFO  MainProcess  MainThread File '/root/py_projects/aihiii/data/doe/doe_big_grid_20230922_154140/channels.parquet' exists
2024-08-06 12:52:43,522     INFO  MainProcess  MainThread Directory '/root/py_projects/aihiii/reports/figures/402_ISO_TimeWindow' exists


In [3]:
def read_data(
    file: Path = CHANNEL_FILE,
    columns: List[str] = RELEVANT_CHANNELS,
    percentiles: List[int] = [5],
    sim_ids: Optional[List[int]] = None,
) -> pd.DataFrame:
    LOG.info("Read data from %s", file)
    filters = [(STR.perc, "in", percentiles)]
    if sim_ids is not None:
        filters.append((STR.id, "in", sim_ids))

    db = pd.read_parquet(path=file, columns=columns, filters=filters).droplevel(STR.perc).apply(pd.to_numeric, downcast="float")

    LOG.info("Got data with shape %s", db.shape)

    return db

In [4]:
def full_channels(channels: List[str] = RELEVANT_CHANNELS, percentiles: List[int] = [95]):
    LOG.info("Read data")
    db: pd.DataFrame = read_data(percentiles=percentiles)

    LOG.info("Calculate percentiles")
    quantile = db.groupby(STR.time).quantile((0.05, 0.25, 0.5, 0.75, 0.95))

    LOG.info("Plot data")
    for channel in tqdm(channels):
        # init
        fig, ax = plt.subplots()

        # plot single signals
        for idx in db.index.get_level_values(STR.id).unique():
            ax.plot(TIME, db.loc[(slice(None), idx), channel], alpha=0.8)

        # plot percentiles
        ax.plot(TIME, quantile.loc[(slice(None), 0.50), channel], label="Median", ls="-", c="black", lw=2)
        ax.plot(TIME, quantile.loc[(slice(None), 0.25), channel], label="IQR", ls="--", c="black", lw=2)
        ax.plot(TIME, quantile.loc[(slice(None), 0.05), channel], label="Percentile 5-95", ls=":", c="black", lw=2)        
        ax.plot(TIME, quantile.loc[(slice(None), 0.75), channel], ls="--", c="black", lw=2)
        ax.plot(TIME, quantile.loc[(slice(None), 0.95), channel], ls=":", c="black", lw=2)

        # style
        ax.legend()
        ax.set_title(f"HIII {percentiles}")
        ax.set_ylabel(channel)
        ax.set_xlabel("Time [ms]")
        ax.grid()

        fig.savefig(FIG_DIR / f"all_ids_{channel}_perc_{'_'.join([str(per) for per in percentiles])}.png")
        plt.close(fig)

    LOG.info("Done")


full_channels()

2024-08-06 12:52:43,572     INFO  MainProcess  MainThread Read data
2024-08-06 12:52:43,573     INFO  MainProcess  MainThread Read data from /root/py_projects/aihiii/data/doe/doe_big_grid_20230922_154140/channels.parquet
2024-08-06 12:52:44,931     INFO  MainProcess  MainThread Got data with shape (4378125, 24)
2024-08-06 12:52:44,932     INFO  MainProcess  MainThread Calculate percentiles
2024-08-06 12:52:47,549     INFO  MainProcess  MainThread Plot data
100%|██████████| 24/24 [03:46<00:00,  9.44s/it]
2024-08-06 12:56:34,167     INFO  MainProcess  MainThread Done


In [5]:
def some_isos(channels: List[str] = RELEVANT_CHANNELS, percentiles: List[int] = [95]) -> Tuple[pd.DataFrame, pd.DataFrame]:
    LOG.info("Read data")
    db: pd.DataFrame = read_data(percentiles=percentiles)

    LOG.info("Calculate percentiles")
    quantile = db.groupby(STR.time).quantile((0.05, 0.25, 0.5, 0.75, 0.95))
    del db

    LOG.info("Calculate")
    cases = list(product(channels, (0.05, 0.25, 0.75, 0.95), ((0, 140), (20, 120), (40, 120), (60, 120), (40, 100), (60, 100))))
    ratings = defaultdict(lambda: defaultdict(dict))
    r2s = defaultdict(lambda: defaultdict(dict))
    for channel, quant, t_range in tqdm(cases):
        # data
        signal_ref = quantile.loc[(slice(*t_range), quant), channel].to_numpy()
        signal_comp = quantile.loc[(slice(*t_range), 0.5), channel].to_numpy()

        # iso
        ratings[channel][quant][t_range] = rating_iso_18571_short(signal_ref=signal_ref, signal_comp=signal_comp)

        # r2
        r2s[channel][quant][t_range] = r2_score(y_true=signal_ref, y_pred=signal_comp)
        r2s[channel][quant][t_range] = r2s[channel][quant][t_range] if r2s[channel][quant][t_range] > 0 else 0

    LOG.info("Convert to DataFrame")
    ratings = pd.DataFrame.from_dict({(i, j): ratings[i][j] for i in ratings.keys() for j in ratings[i].keys()}, orient="index")
    ratings.index.names = ["channel", "quantile"]

    r2s = pd.DataFrame.from_dict({(i, j): r2s[i][j] for i in r2s.keys() for j in r2s[i].keys()}, orient="index")
    r2s.index.names = ["channel", "quantile"]

    LOG.info("Done")

    return ratings, r2s


RATINGS, R2 = some_isos()
RATINGS

2024-08-06 12:56:34,198     INFO  MainProcess  MainThread Read data
2024-08-06 12:56:34,199     INFO  MainProcess  MainThread Read data from /root/py_projects/aihiii/data/doe/doe_big_grid_20230922_154140/channels.parquet
2024-08-06 12:56:35,017     INFO  MainProcess  MainThread Got data with shape (4378125, 24)
2024-08-06 12:56:35,018     INFO  MainProcess  MainThread Calculate percentiles
2024-08-06 12:56:37,476     INFO  MainProcess  MainThread Calculate
100%|██████████| 576/576 [00:17<00:00, 33.28it/s]
2024-08-06 12:56:54,789     INFO  MainProcess  MainThread Convert to DataFrame
2024-08-06 12:56:54,796     INFO  MainProcess  MainThread Done


0         20        40        60        40  \
                                  140       120       120       120       100   
channel            quantile                                                     
03HEADLOC0OCCUDSXD 0.05      0.844179  0.821652  0.767969  0.686299  0.796251   
                   0.25      0.954004  0.944442  0.930811  0.901624  0.943678   
                   0.75      0.969686  0.964689  0.952588  0.932526  0.959134   
                   0.95      0.918450  0.901250  0.864892  0.827817  0.873615   
03HEADLOC0OCCUDSYD 0.05      0.421277  0.390809  0.352436  0.278921  0.328422   
...                               ...       ...       ...       ...       ...   
03FEMRRI00OCCUFOZD 0.95      0.804411  0.794138  0.722545  0.747924  0.679905   
03FEMRLE00OCCUFOZD 0.05      0.854078  0.822747  0.780130  0.811788  0.726136   
                   0.25      0.940817  0.930175  0.914398  0.951497  0.892661   
                   0.75      0.938545  0.926634  0.905777  0.957085  0.883628   
                   0.95      0.807202  0.756696  0.694006  0.714406  0.651756   

                                   60  
                                  100  
channel            quantile            
03HEADLOC0OCCUDSXD 0.05      0.693138  
                   0.25      0.915167  
                   0.75      0.938430  
                   0.95      0.810116  
03HEADLOC0OCCUDSYD 0.05      0.284124  
...                               ...  
03FEMRRI00OCCUFOZD 0.95      0.782608  
03FEMRLE00OCCUFOZD 0.05      0.780211  
                   0.25      0.939606  
                   0.75      0.938701  
                   0.95      0.713261  

[96 rows x 6 columns]

In [6]:
display(RATINGS.groupby("quantile").median())
display(R2.groupby("quantile").median())

,0,20,40,60,40,60
,140,120,120,120,100,100
quantile,,,,,,
0.05,0.632680,0.588887,0.536137,0.463823,0.613823,0.509674
0.25,0.846843,0.810751,0.786569,0.733342,0.779645,0.690392
0.75,0.819922,0.777153,0.741649,0.696768,0.769146,0.680016
0.95,0.592674,0.531578,0.450338,0.402257,0.546082,0.439694


,0,20,40,60,40,60
,140,120,120,120,100,100
quantile,,,,,,
0.05,0.324072,0.174236,0.000000,0.000000,0.052087,0.000000
0.25,0.819856,0.791058,0.681869,0.450987,0.612576,0.347032
0.75,0.815398,0.726628,0.536155,0.406133,0.564894,0.318617
0.95,0.218537,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
display(RATINGS.groupby("quantile").min())
display(R2.groupby("quantile").min())

,0,20,40,60,40,60
,140,120,120,120,100,100
quantile,,,,,,
0.05,0.189207,0.133431,0.096617,0.070115,0.079513,0.068937
0.25,0.157025,0.073196,0.047172,0.034712,0.041090,0.034303
0.75,0.235721,0.209147,0.141454,0.088454,0.138923,0.083879
0.95,0.220609,0.231175,0.166241,0.106361,0.166806,0.100493


,0,20,40,60,40,60
,140,120,120,120,100,100
quantile,,,,,,
0.05,0.0,0.0,0.0,0.0,0.0,0.0
0.25,0.0,0.0,0.0,0.0,0.0,0.0
0.75,0.0,0.0,0.0,0.0,0.0,0.0
0.95,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
display(RATINGS.groupby("quantile").max())
display(R2.groupby("quantile").max())

,0,20,40,60,40,60
,140,120,120,120,100,100
quantile,,,,,,
0.05,0.919218,0.902879,0.881298,0.871347,0.868554,0.816852
0.25,0.979087,0.971891,0.964953,0.951976,0.949855,0.942299
0.75,0.985413,0.981446,0.975098,0.965623,0.964070,0.946313
0.95,0.943227,0.940322,0.924192,0.928199,0.894344,0.874158


,0,20,40,60,40,60
,140,120,120,120,100,100
quantile,,,,,,
0.05,0.954583,0.943207,0.900824,0.917890,0.908158,0.835060
0.25,0.988945,0.984600,0.969887,0.977804,0.974615,0.954126
0.75,0.991772,0.985804,0.971850,0.984106,0.973681,0.925973
0.95,0.965605,0.947436,0.902180,0.939747,0.881254,0.706471


In [9]:
_ = [
    display(pd.concat([RATINGS.loc[(ch, slice(None)), :], R2.loc[(ch, slice(None)), :]], axis=1))
    for ch in RATINGS.index.get_level_values("channel").unique()
]

0         20        40        60        40  \
                                  140       120       120       120       100   
channel            quantile                                                     
03HEADLOC0OCCUDSXD 0.05      0.844179  0.821652  0.767969  0.686299  0.796251   
                   0.25      0.954004  0.944442  0.930811  0.901624  0.943678   
                   0.75      0.969686  0.964689  0.952588  0.932526  0.959134   
                   0.95      0.918450  0.901250  0.864892  0.827817  0.873615   

                                   60        0         20        40        60  \
                                  100       140       120       120       120   
channel            quantile                                                     
03HEADLOC0OCCUDSXD 0.05      0.693138  0.899486  0.840916  0.756498  0.483463   
                   0.25      0.915167  0.980413  0.968969  0.951822  0.894385   
                   0.75      0.938430  0.987438  0.981451  0.969921  0.926682   
                   0.95      0.810116  0.958657  0.941183  0.902180  0.746173   

                                   40        60  
                                  100       100  
channel            quantile                      
03HEADLOC0OCCUDSXD 0.05      0.662391  0.197439  
                   0.25      0.932490  0.837669  
                   0.75      0.958135  0.895934  
                   0.95      0.870222  0.665810

0         20        40        60        40  \
                                  140       120       120       120       100   
channel            quantile                                                     
03HEADLOC0OCCUDSYD 0.05      0.421277  0.390809  0.352436  0.278921  0.328422   
                   0.25      0.442055  0.410639  0.368924  0.317716  0.348671   
                   0.75      0.235721  0.209147  0.141454  0.088454  0.138923   
                   0.95      0.252382  0.231175  0.166241  0.106361  0.166806   

                                   60   0    20   40   60   40   60  
                                  100  140  120  120  120  100  100  
channel            quantile                                          
03HEADLOC0OCCUDSYD 0.05      0.284124  0.0  0.0  0.0  0.0  0.0  0.0  
                   0.25      0.297936  0.0  0.0  0.0  0.0  0.0  0.0  
                   0.75      0.083879  0.0  0.0  0.0  0.0  0.0  0.0  
                   0.95      0.100493  0.0  0.0  0.0  0.0  0.0  0.0

0         20        40        60        40  \
                                  140       120       120       120       100   
channel            quantile                                                     
03HEADLOC0OCCUDSZD 0.05      0.867340  0.886923  0.869380  0.803465  0.868554   
                   0.25      0.945978  0.958789  0.951167  0.926339  0.949855   
                   0.75      0.937706  0.955772  0.947849  0.927056  0.937976   
                   0.95      0.815182  0.868526  0.831067  0.771377  0.809635   

                                   60        0         20        40        60  \
                                  100       140       120       120       120   
channel            quantile                                                     
03HEADLOC0OCCUDSZD 0.05      0.816491  0.936921  0.928562  0.900824  0.808180   
                   0.25      0.940968  0.981081  0.977904  0.969465  0.942423   
                   0.75      0.922574  0.976127  0.970683  0.960643  0.931905   
                   0.95      0.741330  0.857173  0.867641  0.823336  0.701290   

                                   40        60  
                                  100       100  
channel            quantile                      
03HEADLOC0OCCUDSZD 0.05      0.861045  0.706592  
                   0.25      0.954775  0.901933  
                   0.75      0.935539  0.865941  
                   0.95      0.724202  0.436008

0         20        40        60        40  \
                                  140       120       120       120       100   
channel            quantile                                                     
03HEAD0000OCCUACXD 0.05      0.603649  0.556435  0.521747  0.462154  0.613688   
                   0.25      0.827630  0.795746  0.769987  0.691970  0.744410   
                   0.75      0.766474  0.754004  0.723108  0.625386  0.743012   
                   0.95      0.220609  0.274675  0.193942  0.202114  0.409341   

                                   60        0         20        40        60  \
                                  100       140       120       120       120   
channel            quantile                                                     
03HEAD0000OCCUACXD 0.05      0.537889  0.069917  0.000000  0.000000  0.000000   
                   0.25      0.642945  0.778202  0.737526  0.656686  0.442347   
                   0.75      0.617761  0.480805  0.452858  0.309068  0.017954   
                   0.95      0.244933  0.000000  0.000000  0.000000  0.000000   

                                   40        60  
                                  100       100  
channel            quantile                      
03HEAD0000OCCUACXD 0.05      0.145427  0.000000  
                   0.25      0.637866  0.444223  
                   0.75      0.000000  0.000000  
                   0.95      0.000000  0.000000

0         20        40        60        40  \
                                  140       120       120       120       100   
channel            quantile                                                     
03HEAD0000OCCUACYD 0.05      0.362121  0.368755  0.337922  0.280771  0.442408   
                   0.25      0.239313  0.270310  0.236996  0.139679  0.378674   
                   0.75      0.505093  0.531840  0.471365  0.413010  0.409186   
                   0.95      0.498498  0.500836  0.441072  0.392299  0.288765   

                                   60   0    20   40   60   40   60  
                                  100  140  120  120  120  100  100  
channel            quantile                                          
03HEAD0000OCCUACYD 0.05      0.303731  0.0  0.0  0.0  0.0  0.0  0.0  
                   0.25      0.222285  0.0  0.0  0.0  0.0  0.0  0.0  
                   0.75      0.365488  0.0  0.0  0.0  0.0  0.0  0.0  
                   0.95      0.317186  0.0  0.0  0.0  0.0  0.0  0.0

...

0         20        40        60        40  \
                                  140       120       120       120       100   
channel            quantile                                                     
03NECKUP00OCCUFOXD 0.05      0.444980  0.374167  0.278752  0.166478  0.301522   
                   0.25      0.682670  0.641778  0.567250  0.405090  0.537687   
                   0.75      0.711031  0.698666  0.630862  0.503646  0.655123   
                   0.95      0.437024  0.373436  0.292938  0.215385  0.495805   

                                   60        0         20        40        60  \
                                  100       140       120       120       120   
channel            quantile                                                     
03NECKUP00OCCUFOXD 0.05      0.184464  0.000000  0.000000  0.000000  0.000000   
                   0.25      0.366499  0.335333  0.000000  0.000000  0.000000   
                   0.75      0.516621  0.521577  0.496835  0.466512  0.352706   
                   0.95      0.420467  0.000000  0.000000  0.000000  0.000000   

                                   40        60  
                                  100       100  
channel            quantile                      
03NECKUP00OCCUFOXD 0.05      0.000000  0.000000  
                   0.25      0.000000  0.000000  
                   0.75      0.581246  0.392882  
                   0.95      0.013318  0.000000

0         20        40        60        40  \
                                  140       120       120       120       100   
channel            quantile                                                     
03NECKUP00OCCUFOZD 0.05      0.423218  0.387338  0.328111  0.297764  0.424848   
                   0.25      0.783736  0.769247  0.725650  0.663164  0.757112   
                   0.75      0.839187  0.818670  0.788450  0.728230  0.796144   
                   0.95      0.670038  0.630962  0.585186  0.478070  0.592213   

                                   60        0         20        40        60  \
                                  100       140       120       120       120   
channel            quantile                                                     
03NECKUP00OCCUFOZD 0.05      0.231181  0.000000  0.000000  0.000000  0.000000   
                   0.25      0.648802  0.614841  0.513238  0.263941  0.010129   
                   0.75      0.725792  0.812684  0.766691  0.622820  0.383754   
                   0.95      0.468448  0.354624  0.240594  0.000000  0.000000   

                                   40        60  
                                  100       100  
channel            quantile                      
03NECKUP00OCCUFOZD 0.05      0.000000  0.000000  
                   0.25      0.447315  0.237785  
                   0.75      0.673333  0.473596  
                   0.95      0.055981  0.000000

0         20        40        60        40  \
                                  140       120       120       120       100   
channel            quantile                                                     
03NECKUP00OCCUMOYD 0.05      0.307756  0.439522  0.382216  0.280468  0.377931   
                   0.25      0.551270  0.531669  0.484241  0.396687  0.450875   
                   0.75      0.655293  0.619686  0.557000  0.456656  0.655241   
                   0.95      0.428341  0.362248  0.325055  0.260764  0.507978   

                                   60        0        20   40   60        40  \
                                  100       140      120  120  120       100   
channel            quantile                                                    
03NECKUP00OCCUMOYD 0.05      0.249961  0.000000  0.00000  0.0  0.0  0.000000   
                   0.25      0.309922  0.000000  0.00000  0.0  0.0  0.000000   
                   0.75      0.526018  0.340687  0.26091  0.0  0.0  0.372095   
                   0.95      0.406765  0.000000  0.00000  0.0  0.0  0.000000   

                                 60  
                                100  
channel            quantile          
03NECKUP00OCCUMOYD 0.05      0.0000  
                   0.25      0.0000  
                   0.75      0.1881  
                   0.95      0.0000

0         20        40        60        40  \
                                  140       120       120       120       100   
channel            quantile                                                     
03FEMRRI00OCCUFOZD 0.05      0.864420  0.835246  0.785596  0.661336  0.736659   
                   0.25      0.940931  0.933935  0.908359  0.907349  0.887063   
                   0.75      0.936198  0.929468  0.909458  0.917148  0.889720   
                   0.95      0.804411  0.794138  0.722545  0.747924  0.679905   

                                   60        0         20        40        60  \
                                  100       140       120       120       120   
channel            quantile                                                     
03FEMRRI00OCCUFOZD 0.05      0.730842  0.889011  0.824062  0.763099  0.793228   
                   0.25      0.935598  0.968842  0.952588  0.940251  0.966383   
                   0.75      0.932898  0.960988  0.944557  0.932707  0.967050   
                   0.95      0.782608  0.816001  0.754256  0.714217  0.857055   

                                   40        60  
                                  100       100  
channel            quantile                      
03FEMRRI00OCCUFOZD 0.05      0.104173  0.000000  
                   0.25      0.804705  0.868051  
                   0.75      0.821893  0.914757  
                   0.95      0.298286  0.692157

0         20        40        60        40  \
                                  140       120       120       120       100   
channel            quantile                                                     
03FEMRLE00OCCUFOZD 0.05      0.854078  0.822747  0.780130  0.811788  0.726136   
                   0.25      0.940817  0.930175  0.914398  0.951497  0.892661   
                   0.75      0.938545  0.926634  0.905777  0.957085  0.883628   
                   0.95      0.807202  0.756696  0.694006  0.714406  0.651756   

                                   60        0         20        40        60  \
                                  100       140       120       120       120   
channel            quantile                                                     
03FEMRLE00OCCUFOZD 0.05      0.780211  0.860567  0.777400  0.690420  0.811231   
                   0.25      0.939606  0.966391  0.945443  0.925934  0.963135   
                   0.75      0.938701  0.964017  0.942800  0.924862  0.966942   
                   0.95      0.713261  0.800735  0.698598  0.621689  0.781936   

                                   40        60  
                                  100       100  
channel            quantile                      
03FEMRLE00OCCUFOZD 0.05      0.235259  0.333370  
                   0.25      0.800954  0.883976  
                   0.75      0.784343  0.899303  
                   0.95      0.000000  0.417546

In [10]:
from scipy.signal import resample
import matplotlib.colors as mcolors
from scipy.interpolate import CubicSpline


def resampler(n_samples=5, channels: List[str] = RELEVANT_CHANNELS, percentiles: List[int] = [95]):
    LOG.info("Read data")
    cs = sorted(mcolors.TABLEAU_COLORS.values())
    t1 = np.linspace(0, 140, 1401)
    t2 = np.linspace(0, 140, 50)
    rng = np.random.default_rng()
    ids = list(rng.integers(low=0, high=3124, size=n_samples))
    db: pd.DataFrame = read_data(percentiles=percentiles, sim_ids=ids)

    db_ = np.array(np.vsplit(db, n_samples))

    db_resample = CubicSpline(x=t1, y=db_, axis=1)(t2)

    db_back_sampled = CubicSpline(x=t2, y=db_resample, axis=1)(t1)
    LOG.info("Resample from %s to %s and back to %s", db_.shape, db_resample.shape, db_back_sampled.shape)

    for idx, channel in enumerate(channels):
        print(channel, np.sum(np.abs(db_[:, :, idx]-db_back_sampled[:, :, idx]))/(np.max(db_[:, :, idx])-np.min(db_[:, :, idx])))

    LOG.info("Done")


resampler()

2024-08-06 12:56:55,273     INFO  MainProcess  MainThread Read data
2024-08-06 12:56:55,275     INFO  MainProcess  MainThread Read data from /root/py_projects/aihiii/data/doe/doe_big_grid_20230922_154140/channels.parquet
2024-08-06 12:56:55,658     INFO  MainProcess  MainThread Got data with shape (7005, 24)
/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
2024-08-06 12:56:55,672     INFO  MainProcess  MainThread Resample from (5, 1401, 24) to (5, 50, 24) and back to (5, 1401, 24)
2024-08-06 12:56:55,675     INFO  MainProcess  MainThread Done


03HEADLOC0OCCUDSXD 0.08513315313383374
03HEADLOC0OCCUDSYD 0.01962852423877378
03HEADLOC0OCCUDSZD 0.1692611565639017
03HEAD0000OCCUACXD 8.174843759343743
03HEAD0000OCCUACYD 6.457385436249283
03HEAD0000OCCUACZD 11.586831602005148
03CHSTLOC0OCCUDSXD 0.13245359825365996
03CHSTLOC0OCCUDSYD 0.04436790787100297
03CHSTLOC0OCCUDSZD 0.22772520173183353
03CHST0000OCCUDSXD 0.5847470980338872
03CHST0000OCCUACXD 11.016225935921213
03CHST0000OCCUACYD 20.00587143852797
03CHST0000OCCUACZD 10.25279330282399
03PELVLOC0OCCUDSXD 0.14888925737537445
03PELVLOC0OCCUDSYD 0.08649264374818774
03PELVLOC0OCCUDSZD 0.11150175707455748
03PELV0000OCCUACXD 3.555710528649508
03PELV0000OCCUACYD 7.152280083944683
03PELV0000OCCUACZD 3.809827149133544
03NECKUP00OCCUFOXD 5.579983800147533
03NECKUP00OCCUFOZD 11.350342130307887
03NECKUP00OCCUMOYD 4.837343921238097
03FEMRRI00OCCUFOZD 11.103213982147604
03FEMRLE00OCCUFOZD 5.472120585297705


In [9]:
import tensorflow as tf
import numpy as np
from scipy.interpolate import CubicSpline

class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs

    def build(self, input_shape):
        print("###", input_shape)
        pass

    def call(self, inputs):
        t1 = np.linspace(0, 140, 1401)
        t2 = np.linspace(0, 140, self.num_outputs)

        db_resample = CubicSpline(x=t1, y=inputs, axis=1)(t2)

        return db_resample
    
MyDenseLayer(num_outputs=50)(np.random.random((100, 1401, 25))).shape

### (100, 1401, 25)


(100, 50, 25)

In [8]:
def inter(inputs, num_outputs):
    t1 = np.linspace(0, 140, 1401)
    t2 = np.linspace(0, 140, num_outputs)

    return CubicSpline(x=t1, y=inputs, axis=1)(t2)

In [10]:
from time import perf_counter
d = np.random.random((6553, 1401, 43))
print(d.shape)

tic = perf_counter()
_ = inter(d, 50)
print(perf_counter()-tic)

tic = perf_counter()
_ = MyDenseLayer(num_outputs=50)(d)
print(perf_counter()-tic)

(6553, 1401, 43)


: 

In [7]:
import tensorflow as tf
import numpy as np
from scipy.interpolate import CubicSpline

class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs

    def build(self, input_shape):
        print("###", input_shape)
        pass

    def call(self, inputs):
        t1 = np.linspace(0, 140, 1401)
        t2 = np.linspace(0, 140, self.num_outputs)

        db_resample = CubicSpline(x=t1, y=inputs, axis=1)(t2)

        return db_resample
    
MyDenseLayer(num_outputs=50)(np.random.random((100, 1401, 25))).shape

TensorShape([100, 0, 25])